## Farmer Agent Based Model
_Jim Yoon, Emily Rexer, & Travis Thurber_

The Farmer Agent Based Model (`FABMod` eh eh?) integrates with `mosartwmpy` to model adaptive water demand based on crop economics.

TODO: etc etc

<br/>

__TODO:__ consider if something like `nbdev` is a good option for `FABMod`

<br/>

In [121]:
import xarray as xr
import pandas as pd
import numpy as np
import pickle

from pyomo.environ import *
from pyomo.opt import SolverFactory

In [58]:
# These input files come from `mosartwmpy`

# Many-to-many relationships between GRanD Dam IDs and the grid cell IDs that can withdraw water from their reservoirs
dependency_database_path = '../input/reservoirs/dependency_database.parquet'

# Dam/reservoir parameters - in this case used to find which grid cell ID a particular GRanD Dam ID is located on
# reservoir_parameter_path = 'input/reservoirs/grand_reservoir_parameters.nc'
reservoir_parameter_path = '../input/reservoirs/reservoirs.nc'

# Output from `mosartwmpy` for the preceding year
# TODO this should be provided live from wmpy
simulation_output_path = '../output/istarf_validation/istarf_validation_1982*.nc'
# simulation_output_path = '../output/istarf_validation/istarf_validation_1982_01.nc'


In [59]:
# These input files belong to `FABMod`
# TODO they need to be consolidated into a single csv or parquet file

# TODO what really is this?
bias_correction_path = '../mosartwmpy/farmer_abm/hist_avail_bias_correction_20201102.csv'

# This file has the relationships between NLDAS ID and lat/lon
# TODO perhaps we should just add the NLDAS IDs to the existing `mosartwmpy` domain file so that we don't need to use these
nldas_path = '../mosartwmpy/farmer_abm/nldas.txt'

# This file has a list of NLDAS IDs that we care about
nldas_ids_path = '../mosartwmpy/farmer_abm/nldas_ids.p'

# TODO what even is this?
historic_storage_path = '../mosartwmpy/farmer_abm/hist_dependent_storage.csv'


<br/>

`mu` represents the memory decay rate of each agent; higher values indicate faster decay, i.e. 1 means only remember the preceding year.

In [60]:
mu = 0.2

<br/>

Begin by loading the inputs into memory:

In [71]:
# Grid cell to consumable reservoirs
dependency_database = pd.read_parquet(dependency_database_path)

# Placement of reservoirs on the grid
# TODO rather than use strings, need to read variable names from `mosartwmpy` config
reservoir_parameters = xr.open_dataset(reservoir_parameter_path)[['GRAND_ID', 'GRID_CELL_INDEX']].to_dataframe()

# Preceding year of `mosartwmpy` output, subset to the data `FABMod` needs and averaged over the whole year
# TODO rather than use strings, need to read variable names from `mosartwmpy` config
simulation_output = xr.open_mfdataset(simulation_output_path)[[
    'GINDEX', 'WRM_STORAGE', 'WRM_SUPPLY', 'RIVER_DISCHARGE_OVER_LAND_LIQ'
]].mean('time').to_dataframe().reset_index()

# Bias correction
# TODO what is it?
bias_correction = pd.read_csv(bias_correction_path)

# NLDAS IDs to lat/lon
# TODO see above, should just include this in `mosartwmpy` domain
nldas = pd.read_csv(nldas_path)

# Historic storage
# TODO what is it? Do we only need it during the warmup period?
historic_storage = pd.read_csv(historic_storage_path)

# NLDAS IDs we care about
# TODO should just include this in a unified input file that excludes things we don't care about
with open(nldas_ids_path, 'rb') as f:
    nldas_ids = pickle.load(f)
    

<br/>

Merge the reservoir grid cell locations into the dependency database.

In [73]:
# TODO remember to rely on the `mosartwmpy` config file for variable names
dependency_database = dependency_database.merge(reservoir_parameters, how='left', on='GRAND_ID').rename(columns={'GRID_CELL_INDEX': 'RESERVOIR_CELL_INDEX'})

<br/>

Find the total reservoir water volume each grid cell had access to:

In [88]:
# Merge the dependency database with the mean storage at reservoir locations, and aggregate per grid cell
# TODO remember to rely on the `mosartwmpy` config file for variable names
abm_data = dependency_database.merge(simulation_output[[
    'GINDEX', 'WRM_STORAGE'
]], how='left', left_on='RESERVOIR_CELL_INDEX', right_on='GINDEX').groupby('DEPENDENT_CELL_INDEX', as_index=False)[['WRM_STORAGE']].sum().rename(
    columns={'WRM_STORAGE': 'STORAGE_SUM'}
)

<br/>

Find the previous year's mean supply and flow for each grid cell:

In [90]:
# Merge in the mean supply and mean channel outflow from the simulation results per grid cell
abm_data[[
    'WRM_SUPPLY', 'RIVER_DISCHARGE_OVER_LAND_LIQ'
]] =  abm_data[['DEPENDENT_CELL_INDEX']].merge(simulation_output[[
    'GINDEX', 'WRM_SUPPLY', 'RIVER_DISCHARGE_OVER_LAND_LIQ'
]], how='left', left_on='DEPENDENT_CELL_INDEX', right_on='GINDEX')[[
    'WRM_SUPPLY', 'RIVER_DISCHARGE_OVER_LAND_LIQ'
]]

<br/>

Determine the NLDAS IDs for each grid cell:
TODO this should just be part of the `mosartwmpy` domain file and/or the `mosartwmpy` output instead

In [95]:
# Merge the lat/lon
abm_data[[
    'lat', 'lon'
]] = abm_data[['DEPENDENT_CELL_INDEX']].merge(simulation_output[[
    'GINDEX', 'lat', 'lon'
]], how='left', left_on='DEPENDENT_CELL_INDEX', right_on='GINDEX')[[
    'lat', 'lon'
]].round(4)

# Merge the NLDAS_ID
abm_data = nldas[[
    'CENTERX', 'CENTERY', 'NLDAS_ID'
]].merge(abm_data, left_on=['CENTERY', 'CENTERX'], right_on=['lat', 'lon'], how='left')

<br/>

Subselect only the NLDAS IDs we care about:

In [98]:
abm_data = abm_data.loc[abm_data.NLDAS_ID.isin(nldas_ids)]

<br/>

Merge in the other `FABMod` calibration/initial condition data:

TODO: figure out what is actually necessary during warmup vs after warmup (i.e. what data is live?)

In [101]:
# Merge historic storage
abm_data['STORAGE_SUM_OG'] = abm_data[['NLDAS_ID']].merge(historic_storage[[
    'NLDAS_ID','STORAGE_SUM_OG'
]], on='NLDAS_ID', how='left')[['STORAGE_SUM_OG']]

In [102]:
# Merge bias correction, original supply in acreft, and original channel outflow
# TODO what are these
abm_data[[
    'sw_avail_bias_corr','WRM_SUPPLY_acreft_OG','RIVER_DISCHARGE_OVER_LAND_LIQ_OG'
]] = abm_data[['NLDAS_ID']].merge(bias_correction[[
    'NLDAS_ID','sw_avail_bias_corr','WRM_SUPPLY_acreft_OG','RIVER_DISCHARGE_OVER_LAND_LIQ_OG'
]], on='NLDAS_ID', how='left')[[
    'sw_avail_bias_corr','WRM_SUPPLY_acreft_OG','RIVER_DISCHARGE_OVER_LAND_LIQ_OG'
]]
# Rename original supply... TODO whatever for
abm_data['WRM_SUPPLY_acreft_prev'] = abm_data['WRM_SUPPLY_acreft_OG']

<br/>

Zero the missing data:

TODO: is this acceptable or does it mask underlying problems??

In [103]:
abm_data = abm_data.fillna(0)

<br/>

Calculate a "demand factor" for each agent:

TODO: Solicit Jim for better describing what this means

In [104]:
abm_data['demand_factor'] = np.where(
    abm_data['STORAGE_SUM_OG'] > 0,
    abm_data['STORAGE_SUM'] / abm_data['STORAGE_SUM_OG'],
    np.where(
        abm_data['RIVER_DISCHARGE_OVER_LAND_LIQ_OG'] >= 0.1,
        abm_data['RIVER_DISCHARGE_OVER_LAND_LIQ'] / abm_data['RIVER_DISCHARGE_OVER_LAND_LIQ_OG'],
        1
    )
)

<br/>

Calculate new stuff...

TODO: Need Jim's help to understand what each variable means

In [105]:
# TODO
abm_data['WRM_SUPPLY_acreft_newinfo'] = abm_data['demand_factor'] * abm_data['WRM_SUPPLY_acreft_OG']

# TODO
abm_data['WRM_SUPPLY_acreft_updated'] = ((1 - mu) * abm_data['WRM_SUPPLY_acreft_prev']) + (mu * abm_data['WRM_SUPPLY_acreft_newinfo'])

# TODO
abm_data['WRM_SUPPLY_acreft_prev'] = abm_data['WRM_SUPPLY_acreft_updated']

# TODO
abm_data['WRM_SUPPLY_acreft_bias_corr'] = abm_data['WRM_SUPPLY_acreft_updated'] + abm_data['sw_avail_bias_corr']

<br/>

Finally, we have a bias corrected water supply estimate for each agent as a dictionary:

TODO: Ask Jim what this really means

In [126]:
# TODO did we lose the conversion from m3 to acreft somewhere? the bloody hell is an acreft?
water_constraints_by_farm = abm_data.reset_index()['WRM_SUPPLY_acreft_bias_corr'].to_dict()

160      8617.964424
161      4319.254040
162      1923.191471
163      1590.555156
164      3604.955338
            ...     
82007       0.000000
82008       0.000000
82009       0.000000
82010       0.000000
82416       0.000000
Name: WRM_SUPPLY_acreft_bias_corr, Length: 53835, dtype: float64

<br/>

Initialize the ConcreteModel. 

In [131]:
## Read in PMP calibration files
code_path = "../mosartwmpy/farmer_abm/"

data_file=pd.ExcelFile(code_path+"/MOSART_WM_PMP_inputs_20201028.xlsx")
data_profit = data_file.parse("Profit")
water_nirs=data_profit["nir_corrected"]
nirs=dict(water_nirs)

print(f'Loaded PMP calibration files.')

ids = range(538350) # total number of crop and nldas ID combinations
farm_ids = range(53835) # total number of farm agents / nldas IDs
with open(code_path+'/crop_ids_by_farm.p', 'rb') as fp:
    crop_ids_by_farm = pickle.load(fp)
    crop_ids_by_farm_and_constraint = np.copy(crop_ids_by_farm)
with open(code_path+'/max_land_constr_20201102_protocol2.p', 'rb') as fp:
    land_constraints_by_farm = pickle.load(fp)

# Revise to account for removal of "Fodder_Herb category"
crop_ids_by_farm_new = {}
for i in crop_ids_by_farm:
    crop_ids_by_farm_new[i] = crop_ids_by_farm[i][0:10]
crop_ids_by_farm = crop_ids_by_farm_new
crop_ids_by_farm_and_constraint = crop_ids_by_farm_new

# Load gammas and alphas
with open(code_path+'/gammas_new_20201102_protocol2.p', 'rb') as fp:
    gammas = pickle.load(fp)
with open(code_path+'/net_prices_new_20201102_protocol2.p', 'rb') as fp:
    net_prices = pickle.load(fp)

# !JY! replace net_prices with zero value for gammas that equal to zero
for n in range(len(net_prices)):
    if gammas[n] == 0:
        net_prices[n] = 0

x_start_values=dict(enumerate([0.0]*3))

print(f'Loaded constructed model indices, constraints.')

## C.2. 2st stage: Quadratic model included in JWP model simulations
## C.2.a. Constructing model inputs:
##  (repetition to be safe - deepcopy does not work on PYOMO models)
fwm_s = ConcreteModel()
fwm_s.ids = Set(initialize=ids)
fwm_s.farm_ids = Set(initialize=farm_ids)
fwm_s.crop_ids_by_farm = Set(fwm_s.farm_ids, initialize=crop_ids_by_farm)
fwm_s.crop_ids_by_farm_and_constraint = Set(fwm_s.farm_ids, initialize=crop_ids_by_farm_and_constraint)
fwm_s.net_prices = Param(fwm_s.ids, initialize=net_prices, mutable=True)
fwm_s.gammas = Param(fwm_s.ids, initialize=gammas, mutable=True)
fwm_s.land_constraints = Param(fwm_s.farm_ids, initialize=land_constraints_by_farm, mutable=True)
fwm_s.water_constraints = Param(fwm_s.farm_ids, initialize=water_constraints_by_farm, mutable=True) #JY here need to read calculate new water constraints
fwm_s.xs = Var(fwm_s.ids, domain=NonNegativeReals, initialize=x_start_values)
fwm_s.nirs = Param(fwm_s.ids, initialize=nirs, mutable=True)

## C.2.b. 2nd stage model: Constructing functions:
def obj_fun(fwm_s):
    return 0.00001*sum(sum((fwm_s.net_prices[i] * fwm_s.xs[i] - 0.5 * fwm_s.gammas[i] * fwm_s.xs[i] * fwm_s.xs[i]) for i in fwm_s.crop_ids_by_farm[f]) for f in fwm_s.farm_ids)
fwm_s.obj_f = Objective(rule=obj_fun, sense=maximize)


def land_constraint(fwm_s, ff):
    return sum(fwm_s.xs[i] for i in fwm_s.crop_ids_by_farm_and_constraint[ff]) <= fwm_s.land_constraints[ff]
fwm_s.c1 = Constraint(fwm_s.farm_ids, rule=land_constraint)


def water_constraint(fwm_s, ff):
    return sum(fwm_s.xs[i]*fwm_s.nirs[i] for i in fwm_s.crop_ids_by_farm_and_constraint[ff]) <= fwm_s.water_constraints[ff]
fwm_s.c2 = Constraint(fwm_s.farm_ids, rule=water_constraint)

print(f'Constructed pyomo model.')


I have successfully loaded PMP calibration files.
I have loaded constructed model indices, constraints.
I have successfully constructed pyomo model.
Ipopt 3.14.4: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:  1076700
Number of nonzeros in Lagrangian Hessian.............:   239085

Total number of variables............................:   538350
                     variables with only lower bounds:   538350
                variables with lower and upper bounds:   

<br/>

Solve model.

In [ ]:
# Create and run the solver.
try:
    opt = SolverFactory("ipopt", solver_io='nl')
    results = opt.solve(fwm_s, keepfiles=False, tee=True)
    print(results.solver.termination_condition)
    print(f'Solved pyomo model.')
except:
    print(f'Pyomo model solve has failed.')
    # return

<br/>

Store model outputs by writing out demand files for each month.

In [ ]:
## D.1. Storing main model outputs:
            result_xs = dict(fwm_s.xs.get_values())

            # JY store results into a pandas dataframe
            results_pd = data_profit
            results_pd = results_pd.assign(calc_area=result_xs.values())
            results_pd = results_pd.assign(nir=nirs.values())
            results_pd['calc_water_demand'] = results_pd['calc_area'] * results_pd['nir'] / 25583.64
            results_pivot = pd.pivot_table(results_pd, index=['nldas'], values=['calc_water_demand'], aggfunc=np.sum) #JY demand is order of magnitude low, double check calcs

            # JY export results to csv
            results_pd = results_pd[['nldas', 'crop','calc_area']]
            # results_pd.to_csv(output_path+'/abm_results_'+ str(year))
            results_pd.to_csv(output_dir+'/abm_results_'+ str(year))

            # read a sample water demand input file
            file = code_path + '/RCP8.5_GCAM_water_demand_1980_01_copy.nc'
            with netCDF4.Dataset(file, 'r') as nc:
                lat = nc['lat'][:]
                lon = nc['lon'][:]
                demand = nc['totalDemand'][:]

            # read NLDAS grid reference file
            df_grid = pd.read_csv(code_path+'/NLDAS_Grid_Reference.csv')

            df_grid = df_grid[['CENTERX', 'CENTERY', 'NLDAS_X', 'NLDAS_Y', 'NLDAS_ID']]

            df_grid = df_grid.rename(columns={"CENTERX": "longitude", "CENTERY": "latitude"})
            df_grid['longitude'] = df_grid.longitude + 360

            mesh_lon, mesh_lat = np.meshgrid(lon, lat)
            df_nc = pd.DataFrame({'lon':mesh_lon.reshape(-1,order='C'),'lat':mesh_lat.reshape(-1,order='C')})
            df_nc['NLDAS_ID'] = ['x'+str(int((row['lon']-235.0625)/0.125+1))+'y'+str(int((row['lat']-25.0625)/0.125+1)) for _,row in df_nc.iterrows()] 
            df_nc['totalDemand'] = 0

            # use NLDAS_ID as index for both dataframes
            df_nc = df_nc.set_index('NLDAS_ID',drop=False)
            try:
                results_pivot = results.pivot.set_index('nldas',drop=False)
            except KeyError:
                pass

            # read ABM values into df_nc basing on the same index
            df_nc.loc[results_pivot.index,'totalDemand'] = results_pivot.calc_water_demand.values

            # set output dir
            path = self.config.get('simulation.output_path') + '/demand'
            print('Outputting demand files to: ' + path)

            for month in months:
                # str_year = str(year)
                # new_fname = output_path+'/RCP8.5_GCAM_water_demand_'+ str_year + '_' + month + '.nc' # define ABM demand input directory
                # new_fname = path + '/demand_' + str_year + '_' + month + '.nc' # define ABM demand input directory
                new_fname = f"{output_dir}{name}_farmer_abm_demand_{year}_{month}.nc"
                shutil.copyfile(file, new_fname)
                demand_ABM = df_nc.totalDemand.values.reshape(len(lat),len(lon),order='C')
                with netCDF4.Dataset(new_fname,'a') as nc:
                    nc['totalDemand'][:] = np.ma.masked_array(demand_ABM,mask=nc['totalDemand'][:].mask)

            print(f'Written out new demand files for month {month} year {year}.')
            self.processed_years.append(year)